In [1]:
# %reload_ext autoreload
# %autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [40, 30]

In [1]:
import fastai as fai
from fastai.basic_data import DataLoader
import numpy as np
from torch.utils.data import SubsetRandomSampler
import torch 
import pandas as pd
import seaborn as sns
from os import environ
from torch import optim
from bisect import bisect_left
import time
from tqdm import tqdm
from fastai.callbacks import EarlyStoppingCallback
from functools import partial
from fastai import train
from sklearn.metrics import log_loss

from src.data.dataset import *
from src.model.model_bn import *
from src.model.model_bn_ELU import *
from src.model.model_bn_Tanh import *
from src.model.model_bn_LeakyReLU import *
from src.model.model_bn_PReLU import *
from src.model.model_bn_Sigmoid import *
from src.model.classifier_model_bn_Tanh import *
from src.model.classifier_model_bn_ELU import *

In [5]:
optimizer = environ.get('optimizer', 'Adam')
num_workers= int(environ.get('num_workers', '8'))
batch_size=int(environ.get('batch_size', '2048'))
n_epochs=int(environ.get('n_epochs', '500'))
batch_norm = environ.get('batch_norm', 'True') == 'True'
dataset= environ.get('dataset', 'data/speedup_dataset2.pkl')
loss_func = environ.get('loss_func', 'MSE')
log = environ.get('log', 'True') == 'True'
wd = float(environ.get('weight_decay', '0.01'))
cuda_device = environ.get('cuda_device', 'cuda:0')

layers_sizes = list(map(int, environ.get('layers', '300 200 120 80 30').split()))
drops = list(map(float, environ.get('dropouts', '0.2 0.2 0.1 0.1 0.1').split()))
device = torch.device(cuda_device if torch.cuda.is_available() else "cpu")

In [2]:
def train_model(model, criterion, optimizer, dataloader, num_epochs=100):
    since = time.time()
    
    losses = []
    train_loss = 0
    model = model.to(device)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                #scheduler.step()
                model.train()  
            else:
                model.eval()

            running_loss = 0.0
           
            # Iterate over data.
            for inputs, labels in tqdm(dataloader[phase], total=len(dataloader[phase])):       
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)  
                    assert outputs.shape == labels.shape

                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                       # print(loss.item())
                        
            
                # statistics
                running_loss += loss.item()     
                
                #running_corrects += torch.sum((outputs.data - labels.data) < e)/inputs.shape[0]

            epoch_loss = running_loss / len(dataloader[phase])
            
            print('{} Loss: {:.4f}'.format(
               phase, epoch_loss))

            
            if phase == 'val':
                losses.append((train_loss, epoch_loss))
            else:
                train_loss = epoch_loss

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
    
    return losses



def get_results_df(dl, model, log=False):
    df = pd.DataFrame()

    indices = dl.sampler.indices
    
    inputs, targets = dl.dataset[indices]
    names = [dl.dataset.programs[dl.dataset.restricted_program_indexes[i]].name for i in indices]
    sched_names = [dl.dataset.schedules[i].name for i in indices]
    speedups = [dl.dataset.Y_speedups[i] for i in indices]
    inputs = torch.Tensor(inputs)
    
    model.eval()
    preds = model(inputs.to(device))

    interchange, tile, unroll = zip(*[dl.dataset.schedules[index].binary_repr for index in indices])

    preds = preds.cpu().detach().numpy().reshape((-1,))
    targets = targets.reshape((-1,))
    
    if log:
        preds = np.exp(preds*dl.dataset.std + dl.dataset.mean)
        targets = np.exp(targets*dl.dataset.std + dl.dataset.mean)

    df['index'] = indices
    df['name'] = names
    df['sched_name'] = sched_names
    df['prediction'] = preds
    df['target'] = targets
    df['speedup'] = np.array(speedups)
    df['abs_diff'] = np.abs(preds - targets)
    df['APE'] = np.abs(df.target - df.prediction)/df.target * 100
    df['SMAPE'] = 100*np.abs(df.target - df.prediction)/((np.abs(df.target) + np.abs(df.prediction))/2)
    
    df['interchange'] = interchange
    df['tile'] = tile
    df['unroll'] = unroll
    
    return df

#returns classification results dataframe
def get_results_df_classif(dl, model, log=False):
    df = pd.DataFrame()

    indices = dl.sampler.indices
    
    inputs, targets = dl.dataset[indices]
    names = [dl.dataset.programs[dl.dataset.restricted_program_indexes[i]].name for i in indices]
    sched_names = [dl.dataset.schedules[i].name for i in indices]
    speedups = [dl.dataset.Y_speedups[i] for i in indices]
    inputs = torch.Tensor(inputs)
    
    model.eval()
    preds = model(inputs.to(device))

    interchange, tile, unroll = zip(*[dl.dataset.schedules[index].binary_repr for index in indices])

    preds = preds.cpu().detach().numpy().reshape((-1,))
    targets = targets.reshape((-1,))
    
    if log:
        preds = np.exp(preds*dl.dataset.std + dl.dataset.mean)
        targets = np.exp(targets*dl.dataset.std + dl.dataset.mean)

    df['index'] = indices
    df['name'] = names
    df['sched_name'] = sched_names
    df['prediction'] = preds
    df['target'] = targets
    df['speedup'] = np.array(speedups)
    df['abs_diff'] = np.abs(preds - targets)
    df['BCELoss'] = log_loss(df.target , df.prediction)
    
    
    df['interchange'] = interchange
    df['tile'] = tile
    df['unroll'] = unroll
    
    return df

def train_dev_split(dataset, batch_size, num_workers, val_size=10000, test_size=10000, log=False, seed=42):
    
    test_size = test_size
    validation_size = val_size
    ds = DatasetFromPkl(dataset, maxsize=None, log=log)
    
    indices = range(len(ds))
    test_indices, val_indices, train_indices = indices[:test_size], \
                                                indices[test_size:test_size+validation_size], \
                                               indices[test_size+validation_size:]
    train_dl = DataLoader(ds, batch_size=batch_size,
                        sampler=SubsetRandomSampler(train_indices),
                         num_workers=num_workers)

    val_dl = DataLoader(ds, batch_size=batch_size, 
                        sampler=SubsetRandomSampler(val_indices),
                         num_workers=num_workers)
    
    test_dl = DataLoader(ds, batch_size=batch_size, 
                         sampler=SubsetRandomSampler(test_indices),
                         num_workers=num_workers)
    return train_dl, val_dl, test_dl

#loads test, val and train data using filter on speedup and excluding some functions
def train_dev_split_filter(dataset, batch_size, num_workers, val_size=10000, test_size=10000, log=False, seed=42, speedup_lo_bound=0, speedup_up_bound=np.inf, exlude_funcs=[]):
    
    test_size = test_size
    validation_size = val_size
    ds = DatasetFromPkl_Filter(dataset, maxsize=None, log=log, speedup_lo_bound=speedup_lo_bound, speedup_up_bound=speedup_up_bound, exlude_funcs=exlude_funcs)
    
    indices = range(len(ds))
    test_indices, val_indices, train_indices = indices[:test_size], \
                                                indices[test_size:test_size+validation_size], \
                                               indices[test_size+validation_size:]
    train_dl = DataLoader(ds, batch_size=batch_size,
                        sampler=SubsetRandomSampler(train_indices),
                         num_workers=num_workers)

    val_dl = DataLoader(ds, batch_size=batch_size, 
                        sampler=SubsetRandomSampler(val_indices),
                         num_workers=num_workers)
    
    test_dl = DataLoader(ds, batch_size=batch_size, 
                         sampler=SubsetRandomSampler(test_indices),
                         num_workers=num_workers)
    return train_dl, val_dl, test_dl

#loads test, val and train data using a transformation function on speedup and filter function 
def train_dev_split_transform(dataset, batch_size, num_workers, val_size=10000, test_size=10000, log=False, seed=42, filter_func=None, transform_func=None):
    
    test_size = test_size
    validation_size = val_size
    ds = DatasetFromPkl_Transform(dataset, maxsize=None, log=log, filter_func=filter_func, transform_func=transform_func)
    
    indices = range(len(ds))
    test_indices, val_indices, train_indices = indices[:test_size], \
                                                indices[test_size:test_size+validation_size], \
                                               indices[test_size+validation_size:]
    train_dl = DataLoader(ds, batch_size=batch_size,
                        sampler=SubsetRandomSampler(train_indices),
                         num_workers=num_workers)

    val_dl = DataLoader(ds, batch_size=batch_size, 
                        sampler=SubsetRandomSampler(val_indices),
                         num_workers=num_workers)
    
    test_dl = DataLoader(ds, batch_size=batch_size, 
                         sampler=SubsetRandomSampler(test_indices),
                         num_workers=num_workers)
    return train_dl, val_dl, test_dl


def mape_criterion(inputs, targets):
    eps = 1e-5
    return 100*torch.mean(torch.abs(targets - inputs)/(targets+eps))

def smape_criterion(inputs, targets):
    return 100*torch.mean(torch.abs(targets - inputs)/((torch.abs(targets)+torch.abs(inputs))/2))

def rmse_criterion(inputs, targets):
    return torch.sqrt(nn.MSELoss()(inputs, targets))

def mse_criterion(inputs, targets):
    return nn.MSELoss()(inputs, targets)

def bce_criterion(inputs, targets):
    return nn.BCELoss()(inputs, targets)
        
def get_data_with_names(dl):
    dataset = dl.dataset
    names, X, Y = zip(*[(dataset.get_sched_name(index), *dataset[index]) for index in dl.sampler.indices])    
    return names, X, Y

def get_schedule_data(dl, schedule):
    dataset = dl.dataset        
    indices = [index for index in dl.sampler.indices 
                    if np.all(np.array(dataset.schedules[index].binary_repr) + np.array(schedule) != 1)]    
    return indices
    
    
def get_data_with_prog_names(dl):
    dataset = dl.dataset
    names, X, Y = zip(*[(dataset.get_prog_name(index), *dataset[index]) for index in dl.sampler.indices])
    
    return names, X, Y

def joint_plot(df, title, val_range=list(range(-1, 15))):
    ax = sns.jointplot('target', 'prediction', df, ).ax_joint
    plt.suptitle(title)
    _ = ax.set_xticks(val_range)
    _ = ax.set_yticks(val_range)
    _ = ax.plot(val_range, val_range, ':k')
    
class NameGetter(object):
    def __init__(self, dataset):
        self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        return self.dataset.get_prog_name(index)
    
def get_program_data(dl, prog_name):
    dataset = dl.dataset
    name_g = NameGetter(dataset)
    
    index1 = bisect_left(name_g, prog_name, 0)
    index2 = bisect_right(name_g, prog_name, index1)
    
    X, Y = zip(*[dataset[index] for index in range(index1, index2)])
   
    return torch.Tensor(X), torch.Tensor(Y)

def joint_plot_one_program(dl, prog_name, model):
    model.eval()
    
    X, Y = get_program_data(dl, prog_name)
    
    Y_hat = model(X.to(device))
    df = pd.DataFrame()
    df['prediction'] = np.array(Y_hat.view(-1,))
    df['target'] = np.array(Y)
    
    joint_plot(df, prog_name)
    
def joint_plot_one_schedule(dl, schedule, model, log=False):
    indices = get_schedule_data(dl, schedule)
    
    X, Y = dl.dataset[indices]
    X, Y = torch.Tensor(X), torch.Tensor(Y)
    
    Y_hat = model(X.to(device))

    
    df = pd.DataFrame()
    df['prediction'] = np.array(Y_hat.view(-1,))
    df['target'] = np.array(Y)
    
    joint_plot(df, schedule)
    
    
    
    